# Real-world Data Wrangling

In this project, you will apply the skills you acquired in the course to gather and wrangle real-world data with two datasets of your choice.

You will retrieve and extract the data, assess the data programmatically and visually, accross elements of data quality and structure, and implement a cleaning strategy for the data. You will then store the updated data into your selected database/data store, combine the data, and answer a research question with the datasets.

Throughout the process, you are expected to:

1. Explain your decisions towards methods used for gathering, assessing, cleaning, storing, and answering the research question
2. Write code comments so your code is more readable

## 1. Gather data

In this section, you will extract data using two different data gathering methods and combine the data. Use at least two different types of data-gathering methods.

### **1.1.** Problem Statement

I'd like to build the foundation for a historical weather dataset that is focused on Lander, Wyoming weather patterns. I'd like to identify how Climate Change has impacted my area in a particular.
The challenge will be getting long term historical data combined with the most recent detailed data for in depth analysis. This can later be combined with emission data from the Department of Transportation, or Water Quality data from the State of Wyoming.
The goal will be to build this into a normalized database and add re-usable methods for fetching and storing the data so the reports could be expanded to other locations.

### **1.2.** Gather at least two datasets using two different data gathering methods

List of data gathering methods:

- Download data manually
- Programmatically downloading files
- Gather data by accessing APIs
- Gather and extract data from HTML files using BeautifulSoup
- Extract data from a SQL database

Each dataset must have at least two variables, and have greater than 500 data samples within each dataset.

For each dataset, briefly describe why you picked the dataset and the gathering method (2-3 full sentences), including the names and significance of the variables in the dataset. Show your work (e.g., if using an API to download the data, please include a snippet of your code). 

Load the dataset programmtically into this notebook.

#### **Hourly Weather Data**

Type: Tabular records for hourly weather reporting in Lander Wyoming.

Method: Gather data by accessing the Open Meteo Historical Weather Data API.

The first dataset will come from the [Open Meteo API](https://open-meteo.com/en/docs/historical-forecast-api). Source information for this dataset can be found [here](https://open-meteo.com/en/docs/historical-forecast-api#data_sources).
This archival API only has access to 2016 and beyond.

Hourly Weather dataset variables:
* latitude: float - latitude of the location
* longitude: float - longitude of the location
* start_date: datetime - start of hour (America/Denver)
* end_date: datetime - end of hour (America/Denver)
* temperature_2m: float - average temperature aggregated by 2m intervals (fahrenheit)
* precipitation: float - sum of precipitation in (inch)
* wind_speed_10m: float - average wind speed aggregated by 10m intervals (mph)

In [ ]:
from main import migrate, import_weather_data
from models import DailyWeatherRecord, NOAAStationMonthlySummary
import pandas as pd
from constants import ENGINE
import seaborn as sns
import matplotlib.pyplot as plt
from sqlalchemy import insert

IMPORT = False
pd.options.mode.copy_on_write = True

In [ ]:
# Build the weather data set
migrate()

In [ ]:
# Import the weather data from open meteo API archive
if IMPORT:
    import_weather_data(start_date="2017-01-01", end_date="2025-08-01")

In [ ]:
df_hourly = pd.read_sql_table("hourly_weather", ENGINE)

In [ ]:
df_hourly.shape

#### NOAA Monthly Summary

Type: CSV File


Method: From this [link](https://www.ncei.noaa.gov/access/search/data-search/global-summary-of-the-month), filter the location to Lander Wyoming and download the full data set for 'LANDER AIRPORT, WY US (USW00024021.csv)'

This DataSet was gathered from the National Centers for Environmental Information (NOAA), and is representative of the Lander Wyoming Weather data since 1948.
More specifically this source is from the [Global Summary of the Month](https://www.ncei.noaa.gov/access/search/data-search/global-summary-of-the-month) aggregate dataset.
This data will be parsed and paired down to a usable dataset, and later stored to then compare with current trends from Data Set 1.

The dataset has 110 columns to choose from, all of which use a shorthand so [this PDF](https://www.ncei.noaa.gov/pub/data/cdo/documentation/GSOM_documentation.pdf) can be used as a reference. Otherwise, these are the fields of interest that will be stored.



In [ ]:
df_noaa = pd.read_csv("./USWGlobalSummaryOfTheMonth.csv")

In [ ]:
df_noaa.head()

## Dataset 2 fields of interest
* DATE - datetime
* LATITUDE - float
* LONGITUDE - float
* NAME - string
* ADPT - Monthly Average Dew Point Temperature
* AWND - Monthly Average Wind Speed
* CDSD - Cooling Degree Days (season-to-date)
* DP01 - Number of days with >= 0.01 inch/0.254 millimeter in the month
* DP1X - Number of days with >= 1.00 inch/25.4 millimeters in the month
* DSND - Number of days with snow depth >= 1 inch/25 millimeters
* DSNW - Number of days with snowfall >= 1 inch/25 millimeters
* DT00 Number of days with maximum temperature <= 0 degrees Fahrenheit/-17.8 degrees Celsius
* DT32 - Number of days with minimum temperature <= 32 degrees Fahrenheit/0 degrees Celsius
* DX32 - Number of days with maximum temperature <= 32 degrees Fahrenheit/0 degrees Celsius
* DX70 - Number of days with maximum temperature >= 70 degrees Fahrenheit/21.1 degrees Celsius
* DX90 - Number of days with maximum temperature >= 90 degrees Fahrenheit/32.2 degrees Celsius
* EMNT - Extreme minimum temperature for month. Lowest daily minimum temperature for the
month
* EMXP - Highest daily total of precipitation in the month. Given in inches or millimeters depending
on user specification
* EMXT - Extreme maximum temperature for month. Highest daily maximum temperature for the
month. Given in Celsius or Fahrenheit depending on user specification
* PRCP - Total Monthly Precipitation
* PSUN - Monthly Average of the daily percents of possible sunshine
* SNOW - Total Monthly Snowfall
* TAVG - Average Monthly Temperature
* TMAX - Monthly Maximum Temperature. Average of daily maximum temperature
* TMIN - Monthly Minimum Temperature. Average of daily minimum temperature
* TSUN - Daily total sunshine in minutes

## 2. Assess data

Assess the data according to data quality and tidiness metrics using the report below.

List **two** data quality issues and **two** tidiness issues. Assess each data issue visually **and** programmatically, then briefly describe the issue you find.  **Make sure you include justifications for the methods you use for the assessment.**

### Quality Issue 1:
The DATE column is in the incorrect datatype, and it would be beneficial to have a year and month column for better grouping analysis
Issue and justification:
Not having values for these fields means they will not be useful for analysis

In [ ]:
# The DF Hourly data is in good shap for null values
df_hourly.isnull().sum()
df_hourly.isna().sum()

In [ ]:
# The date format is YYYY-MM and is an object type, in order to be able to sort dates there should be a year and month column
df_noaa["DATE"].dtype

### Quality Issue 2:
The NOAA Dataset has a large number of null values for the chosen location. Sort through these and drop ones that are mostly null >=50%

In [ ]:
# The NOAA dataset has a large number of null values
df_noaa.isnull().sum().sort_values(ascending=False)
# I suspect the missing values may be related to dates

### Tidiness Issue 1:
Given that we know the location is Lander Wyoming we can drop that data and replace it with location id = 1 prior to storing

In [ ]:
# FILL IN - Inspecting the dataframe visually
df_noaa[['NAME', 'LATITUDE', 'LONGITUDE', 'STATION', 'ELEVATION']].value_counts()

### Tidiness Issue 2:
The incoming data is setup as hourly data, this is great for granularity, but to compare to the monthly summary we will need to build up a daily and weekly set

In [ ]:
df_hourly.describe()

In [ ]:
df_hourly.info()

## 3. Clean data
Clean the data to solve the 4 issues corresponding to data quality and tidiness found in the assessing step. **Make sure you include justifications for your cleaning decisions.**

After the cleaning for each issue, please use **either** the visually or programatical method to validate the cleaning was succesful.

At this stage, you are also expected to remove variables that are unnecessary for your analysis and combine your datasets. Depending on your datasets, you may choose to perform variable combination and elimination before or after the cleaning stage. Your dataset must have **at least** 4 variables after combining the data.

In [ ]:
df_hourly_copy = df_hourly.copy()
df_noaa_copy = df_noaa.copy()

### **Quality Issue 1:**
Setup as a proper date format and abstract out a year and month column for later grouping

In [ ]:
df_noaa_copy.loc[:, "year"] = pd.Series(
    [dt[1].split("-")[0] for dt in df_noaa_copy["DATE"].items()]
)
df_noaa_copy.loc[:, "month"] = pd.Series(
    [dt[1].split("-")[1] for dt in df_noaa_copy["DATE"].items()]
)
df_noaa_copy.head()

In [ ]:
# Ensure data integrity prior to conversion
int_cols = ['DP01','DP10','DP1X','DSND','DSNW','DT00','DT32','DX32','DX70','DX90','DYFG','DYHF','DYNT','DYSD','DYSN','DYTS','DYXP','DYXT','WDF2','WDF5']
float_cols = ['CDSD','CLDD','EMNT','EMSD','EMSN','EMXP','EMXT','HDSD','HTDD','PRCP','SNOW','TAVG','TMAX','TMIN','WSF2','WSF5']

for col in int_cols:
    if col in df_noaa_copy.columns:
        df_noaa_copy[col] = pd.to_numeric(df_noaa_copy[col], errors='coerce').astype('Int64')

for col in float_cols:
    if col in df_noaa_copy.columns:
        df_noaa_copy[col] = pd.to_numeric(df_noaa_copy[col], errors='coerce').astype(float)

# Ensure date is datetime64[ns] representing the first day of the month
df_noaa_copy['date'] = pd.to_datetime(df_noaa_copy['DATE'], format='%Y-%m', errors='raise')

# Inspect dtypes after coercion
df_noaa_copy.info()

### **Quality Issue 2**
Identifying null columns and dropping ones that exceed a reasonable threshold. If a pattern develops by year we could shorten the time span for comparision.

In [ ]:
# 1) Ensure we filter out the _ATTRIBUTE columns to shorten the width of this df
df_noaa_copy = df_noaa_copy[[col for col in df_noaa_copy.columns if not col.endswith('_ATTRIBUTES')]]
# Count total rows per year (denominator)
rows_per_year = df_noaa_copy.groupby("year").size()

# Sum of nulls per year by selected columns
nulls_by_year = df_noaa_copy.groupby("year")[df_noaa_copy.columns].apply(
    lambda g: g.isnull().sum()
)

# Convert to proportions
null_proportion_by_year = (nulls_by_year.div(rows_per_year, axis=0) * 100).round(1)


In [ ]:
plt.figure(figsize=(18, 9))
ax = sns.heatmap(
    null_proportion_by_year,
    annot=False,
    cmap='Greens',
    vmin=0, vmax=100,
    linewidths=0.5, linecolor='white',
    cbar_kws={'label': '% null', 'format': '%.0f%%'}
)
plt.title('Proportion Nulls by Year (Top Null Columns)')
plt.ylabel('Year')
plt.xlabel('Column')
plt.xticks(rotation=45, ha='right', fontsize=9)
plt.yticks(fontsize=10)
plt.tight_layout()
plt.show()

In [ ]:
# Using the heatmap above we can visually see where some fields are null and at which year they are concentrated.
# To clean up this chart we filter out columns that won't be useful for analysis, and include fields of interest
try:
    df_noaa_copy.drop(['ADPT', 'ASLP', 'ASTP', 'AWBT', 'AWND', 'PSUN', 'RHAV', 'RHMN', 'RHMX', 'TSUN', 'WDF1', 'WDFG', 'WDFM', 'WSF1', 'WSFG', 'WSFM'], axis=1, inplace=True)
except KeyError:
    # Already dropped these columns
    pass
rows_per_year = df_noaa_copy.groupby("year").size()
nulls_by_year = df_noaa_copy.groupby("year")[df_noaa_copy.columns].apply(
    lambda g: g.isnull().sum()
)
null_proportion_by_year = (nulls_by_year.div(rows_per_year, axis=0) * 100).round(1)

In [ ]:
plt.figure(figsize=(18, 9))
ax = sns.heatmap(
    null_proportion_by_year,
    annot=False,
    cmap='Greens',
    vmin=0, vmax=100,
    linewidths=0.5, linecolor='white',
    cbar_kws={'label': '% null', 'format': '%.0f%%'}
)
plt.title('Proportion Nulls by Year (Top Null Columns)')
plt.ylabel('Year')
plt.xlabel('Column')
plt.xticks(rotation=45, ha='right', fontsize=9)
plt.yticks(fontsize=10)
plt.tight_layout()
plt.show()

Now the Heatmap is looking a lot cleaner. We will have to keep in mind there are some gaps with the WDF fields, but since it is bound by date we can work around that.

### **Tidiness Issue 1**
Prior to storing we will want to bring the location data into normal form via the Location table found in models.py

In [ ]:
df_noaa_copy.info()

In [ ]:
# We can remove the station
current_columns = df_noaa_copy.columns
if 'STATION' in current_columns:
    df_noaa_copy.drop('STATION', axis=1, inplace=True)
if 'LONGITUDE' in current_columns:
    df_noaa_copy.drop('LONGITUDE', axis=1, inplace=True)
if 'LATITUDE' in current_columns:
    df_noaa_copy.drop('LATITUDE', axis=1, inplace=True)
if 'NAME' in current_columns:
    df_noaa_copy.drop('NAME', axis=1, inplace=True)
if 'ELEVATION' in current_columns:
    df_noaa_copy.drop('ELEVATION', axis=1, inplace=True)
if 'location_id' not in current_columns:
    df_noaa_copy.loc[:, 'location_id'] = 1
df_noaa_copy.info()

Justification: By simplifying down to a normal form location table we can expand this to join to other data sets by location and date in the future.

### **Tidiness Issue 2**
In order to compare monthly data we will need to build up a weekly and then monthly summary with the Open Meteo dataset.

In [ ]:
# Clear out any existing records in the daily_weather table
DailyWeatherRecord.__table__.drop(ENGINE, checkfirst=True)
DailyWeatherRecord.__table__.create(ENGINE, checkfirst=True)
df_hourly_copy

In [ ]:
df_hourly_copy["day"] = df_hourly_copy["date"].dt.normalize()
df_hourly_copy.head()
# Add in proper date formatting

In [ ]:
# Group by day and compute aggregates
df_agg_daily = (
    df_hourly_copy.groupby("day").agg(
        average_temperature=("temperature", "mean"),
        min_temperature=("temperature", "min"),
        max_temperature=("temperature", "max"),
        average_wind_speed=("wind_speed", "mean"),
        min_wind_speed=("wind_speed", "min"),
        max_wind_speed=("wind_speed", "max"),
        precipitation_sum=("precipitation", "sum"),
        precipitation_min=("precipitation", "min"),
        precipitation_max=("precipitation", "max"),
    )
    .reset_index()
    .rename(columns={"day": "date_time"})
)
df_agg_daily.info()

In [ ]:
# Add calendar columns and location_id
df_agg_daily["month"] = df_agg_daily["date_time"].dt.month.astype(int)
df_agg_daily["day_of_month"] = df_agg_daily["date_time"].dt.day.astype(int)
df_agg_daily["year"] = df_agg_daily["date_time"].dt.year.astype(int)
df_agg_daily["location_id"] = 1 # Current default location is 1 for Lander, Wyoming

In [ ]:
df_agg_daily.info()

### **Remove unnecessary variables and combine datasets**

Depending on the datasets, you can also peform the combination before the cleaning steps.

In [ ]:
# Unnecessary variables and fields were removed prior to this step

## 4. Update your data store
Update your local database/data store with the cleaned data, following best practices for storing your cleaned data:

- Must maintain different instances / versions of data (raw and cleaned data)
- Must name the dataset files informatively
- Ensure both the raw and cleaned data is saved to your database/data store

In [ ]:
# Ensure column ordering matches models.py file
agg_df = df_agg_daily[
    [
        "location_id",
        "date_time",
        "month",
        "day_of_month",
        "year",
        "average_temperature",
        "min_temperature",
        "max_temperature",
        "average_wind_speed",
        "min_wind_speed",
        "max_wind_speed",
        "precipitation_sum",
        "precipitation_min",
        "precipitation_max",
    ]
]

records = agg_df.to_dict(orient="records")
stmt = insert(DailyWeatherRecord)
with ENGINE.begin() as conn:
    for record in records:
        conn.execute(stmt, record)
    conn.commit()

In [ ]:
NOAAStationMonthlySummary.__table__.drop(ENGINE, checkfirst=True)
NOAAStationMonthlySummary.__table__.create(ENGINE, checkfirst=True)
df_noaa_final = df_noaa_copy[
    ['location_id', 'date', 'CDSD', 'CLDD', 'DP01', 'DP10', 'DP1X', 'DSND', 'DSNW', 'DT00',
       'DT32', 'DX32', 'DX70', 'DX90', 'DYFG', 'DYHF', 'DYNT', 'DYSD', 'DYSN',
       'DYTS', 'DYXP', 'DYXT', 'EMNT', 'EMSD', 'EMSN', 'EMXP', 'EMXT', 'HDSD',
       'HTDD', 'PRCP', 'SNOW', 'TAVG', 'TMAX', 'TMIN', 'WDF2', 'WDF5', 'WSF2',
       'WSF5']
]

records = df_noaa_final.to_dict(orient="records")
stmt = insert(NOAAStationMonthlySummary)
with ENGINE.begin() as conn:
    for record in records:
        conn.execute(stmt, record)
    conn.commit()

## 5. Answer the research question

### **5.1:** Define and answer the research question 
Going back to the problem statement in step 1, use the cleaned data to answer the question you raised. Produce **at least** two visualizations using the cleaned data and explain how they help you answer the question.

*Research question:* FILL IN from answer to Step 1

In [ ]:
# Visual 1 - Year-over-Year change in TMAX (annual mean)
_df_noaa = pd.read_sql_table('noaa_monthly_summary', ENGINE)
# Ensure date is datetime
a = _df_noaa
if not pd.api.types.is_datetime64_any_dtype(a.get('date')):
    a['date'] = pd.to_datetime(a['date'])
# Compute annual mean TMAX
a['year'] = a['date'].dt.year.astype(int)
annual_tmax = (
    a.groupby('year', as_index=False)['TMAX']
     .mean()
     .rename(columns={'TMAX': 'tmax_mean'})
)
# Year-over-year absolute change
annual_tmax['yoy_change'] = annual_tmax['tmax_mean'].diff()
# Plot YoY change as a bar chart with up/down colors
colors = ['green' if v >= 0 else 'red' for v in annual_tmax['yoy_change'].fillna(0)]
ax = annual_tmax.plot(x='year', y='yoy_change', kind='bar', color=colors, legend=False, figsize=(18, 5))
ax.set_title('Year-over-Year Change in TMAX (Annual Mean)')
ax.set_xlabel('Year')
ax.set_ylabel('Change in TMAX')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

*Answer to research question:*
As we can see there is some variability year over year, but the average maximum temperature has been trending upwards in Lander Wyoming. Next let's see by how much over each decade.

In [ ]:
# Visual 2 - Annual average temperature trend (TAVG) with 10-year rolling mean and linear trend
annual_tavg = (
    _df_noaa.groupby('year', as_index=False)['TAVG']
      .mean()
      .rename(columns={'TAVG': 'tavg_mean'})
      .sort_values('year')
)
# 10-year rolling mean (centered)
annual_tavg['roll10'] = annual_tavg['tavg_mean'].rolling(10, min_periods=5, center=True).mean()
# Linear trend
import numpy as np
x = annual_tavg['year'].to_numpy()
y = annual_tavg['tavg_mean'].to_numpy()
if len(annual_tavg) >= 2:
    coef = np.polyfit(x, y, 1)
    trend_fn = np.poly1d(coef)
    annual_tavg['trend'] = trend_fn(x)
    slope_decade = coef[0] * 10.0
else:
    annual_tavg['trend'] = np.nan
    slope_decade = float('nan')
# Plot
plt.figure(figsize=(18, 6))
plt.plot(annual_tavg['year'], annual_tavg['tavg_mean'], color='lightgray', linewidth=2, label='Annual mean (TAVG)')
plt.plot(annual_tavg['year'], annual_tavg['roll10'], color='steelblue', linewidth=3, label='10-yr rolling mean')
if np.isfinite(slope_decade):
    plt.plot(annual_tavg['year'], annual_tavg['trend'], color='crimson', linestyle='--', linewidth=2, label='Linear trend')
    plt.title('Annual Average Temperature (TAVG) with 10-year Rolling Mean and Linear Trend')
else:
    plt.title('Annual Average Temperature (TAVG) with 10-year Rolling Mean')
plt.xlabel('Year')
plt.ylabel('Temperature (°C)')
plt.legend()
plt.grid(alpha=0.2)
plt.tight_layout()
if np.isfinite(slope_decade):
    print(f'Estimated warming trend: {slope_decade:.2f} °C per decade')

*Answer to research question:*
To further suggest the increase in temperature year over year we can see in the above chart that the average temperature has been trending hotter by around 0.1 degrees C.

### **5.2:** Reflection
In 2-4 sentences, if you had more time to complete the project, what actions would you take? For example, which data quality and structural issues would you look into further, and what research questions would you further explore?

*Answer:*
Given more time I would like to investigate the trends as of late, and how they compare to events around the world. For example hurricanes, Cloud Seeding, and changes in emissions output or policy.
Given more time I would like to investigate further trends in the wide assortment of variables present in the Open Meteo, and NOAA data sets. By having these sets parsed aggregated and in a local database I willb e able to build on this project as time goes on.